In [3]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
## BADD-- supressing f1 score ill-defined and being set to 0.0 in labels with no true samples warning
import warnings
warnings.filterwarnings("ignore")

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np

In [7]:
import  datatset
import utils
from torch.utils.data import DataLoader

# Data

In [8]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
# TRAIN_PATH = Path("/home/ubuntu/data-1/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [9]:
def over_sample_class(x):
    if len(x) < 20:
        return x.sample(n=20, replace=True) 
    return x

In [10]:
def sample_class(x):
    if len(x) > 100:
        return x

In [11]:
classes_map = utils.load_json(DATA_DIR/"classes-map.json")

In [12]:
len(df_train), len(df_train["category_id"].value_counts())

(1030747, 32093)

### Over sample and apply data augmentation

In [14]:
# Over Sample Class
new_df = df_train.groupby('category_id').apply(over_sample_class).reset_index(drop = True)

In [17]:
# apply data augmentation to labels with less then 21 exampels
labels_aug = counts.keys()[counts < 21] # Get labels that have less then 9 exmaples
augment = new_df["category_id"].isin(labels_aug) # Get list of items to apply aug

In [18]:
new_df["augment"] = augment #np.zeros(len(new_df)) == 1 # set all augemntaion to zero

In [19]:
np.sum(new_df.augment)

0

### Update Labels

In [21]:
# Update class labels
new_labels = [classes_map[str(i)] for i in new_df["category_id"]]
new_df["new_labels"] = new_labels

In [22]:
num_calsses = len(classes)

In [23]:
num_calsses

32093

In [24]:
from tqdm import tqdm_notebook

# there is a better way to do this
def sample_for_validation(df, n=4):   
    validation_set = {}
    bool_arr = []
    for i,row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
        if validation_set.get(row["new_labels"], 0) == n:
            bool_arr.append(False) 
        else:
            validation_set[row["new_labels"]] = validation_set.get(row["new_labels"], 0) + 1
            bool_arr.append(True) 

        
    return bool_arr

### Split into train and validation

In [26]:
new_df["is_valid"] = sample(new_df, n=4)

In [27]:
train_df, valid_df = new_df[~new_df["is_valid"]] , new_df[new_df["is_valid"]]

In [28]:
# len(train_df), len(valid_df)
# (703500, 14766)

In [29]:
len(train_df), len(valid_df)

(703500, 14766)

In [30]:
valid_df.to_csv(TRAIN_PATH/"validation.csv")

In [31]:
ds = datatset.HerbDataSet(train_df, TRAIN_PATH, 256, label="new_labels")

In [32]:
data_loader = DataLoader(ds, batch_size=64, shuffle=True)

# Model

In [34]:
import model
import torchvision.models as models

In [35]:
m = model.get_model(models.resnet50(pretrained=True), p1=0.2, p2=0.2, device="gpu") # no drop out, trying to over fit

In [36]:
# m.load('/home/ubuntu/data-1/nybg2020/train/models/model-11596.pth')

In [37]:
import torch.nn as nn
import torch

In [38]:
assert m.fc[4].out_features == num_calsses

# Tracking

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn

In [1]:
RUN = 


# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/'logs/{RUN}')

SyntaxError: invalid syntax (<ipython-input-1-4ea03e816920>, line 1)

In [ ]:
model_path=TRAIN_PATH/f'models-{RUN}/

In [ ]:
assert model_path.exists()

In [41]:
import metrics
import train

In [42]:
acc =  metrics.Accuracy()
f1 = metrics.F1()
manager = metrics.MetricManager([acc, f1], writer)

# Train

In [44]:
criterion = nn.CrossEntropyLoss()

In [45]:
layer_lrs = [
    {'params': m.layer1.parameters(), 'lr': 1e-6},
    {'params': m.layer2.parameters(), 'lr': 1e-6},
    {'params': m.layer3.parameters(), 'lr': 1e-5},
    {'params': m.layer4.parameters(), 'lr': 1e-4},
    {'params': m.fc.parameters(), 'lr': 1e-3}
]

In [46]:
optimizer = torch.optim.Adam(layer_lrs, lr=1e-3) # only training final layers

AssertionError: 

In [49]:
trainer = train.Training(manager, criterion, optimizer, data_loader, 5, m, model_path)

In [50]:
! export CUDA_LAUNCH_BLOCKING=1

In [ ]:
trainer.run()

Epoch 0/5



Epoch 1/5



Epoch 2/5
